In [11]:
# in this notebook varya is trying to export the median point (from the precipitation) from HMA - regions 
# to a dataframe and then later csv file to use as a forsing for testrun of sedcas model 

# take the locations determined in a different notebook (also stored in some csv file already
# find that location in the HMA climate from the server 
# convert to csv 
# save 


In [12]:
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import geopandas as gpd
import os

In [13]:
# read locations file - for annual sum precipitations: 
locations_pr = pd.read_table('out/median_annual_precip_regions.csv', sep = ',',index_col =0)

# read locations file - for annial median temp:
locations_t2m = pd.read_table('out/annual_median_t2m_regions.csv', sep = ',', index_col = 0)

# read hma polygons file to a df:
hma = gpd.read_file('HMA_regions/HMA_regions.shp')
hma_df = pd.DataFrame(hma) 

In [14]:
# locations_pr.head()
# locations_t2m

In [16]:
%%time
# ----------- read climate data: ----------- 

# temperatures
temps = xr.open_mfdataset("/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/2m-temperature/*.nc")

# precipitations
precip = xr.open_mfdataset("/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/total-precipitation/*.nc")

# total cloud cover 
clouds = xr.open_mfdataset("/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/total-cloud-cover/*.nc")

# incoming radiation 
radiation = xr.open_mfdataset("/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/surface-solar-radiation-downwards/*.nc")


# path for output timeseries
path_output = "/Users/varyabazilova/Desktop/hma_regions/out/timeseries"


/Users/varyabazilova/opt/anaconda3/envs/project1/lib/python3.7/site-packages/scipy/io/netcdf.py:312: RuntimeWarning: Cannot close a netcdf_file opened with mmap=True, when netcdf_variables or arrays referring to its data still exist. All data arrays obtained from such files refer directly to data on disk, and must be copied before the file can be cleanly closed. (See netcdf_file docstring for more information on mmap.)
  ), category=RuntimeWarning)
/Users/varyabazilova/opt/anaconda3/envs/project1/lib/python3.7/site-packages/scipy/io/netcdf.py:312: RuntimeWarning: Cannot close a netcdf_file opened with mmap=True, when netcdf_variables or arrays referring to its data still exist. All data arrays obtained from such files refer directly to data on disk, and must be copied before the file can be cleanly closed. (See netcdf_file docstring for more information on mmap.)
  ), category=RuntimeWarning)
/Users/varyabazilova/opt/anaconda3/envs/project1/lib/python3.7/site-packages/scipy/io/netcdf.p

CPU times: user 4.22 s, sys: 11.5 s, total: 15.7 s
Wall time: 1min 3s


In [ ]:
len(temps.t2m.values)

In [9]:
# ----------- convert units: ----------- 

# climate = xr.open_mfdataset('*.nc', decode_coords="all")
# cnange units (and overwrite the metadata with the units after)

# ------ UNITS ------

#convert temperature K to C
temps['t2m']=temps.t2m-273.15
# precipotation m to mm 
precip['tp']=precip.tp * 1000
# radiation j/m2 to w/m2
# SSR [W/m2] = SSR [J/m^2] / (3600 seconds)
radiation['ssrd'] = radiation.ssrd / 3600

### (1) select coordinates 

In [5]:
# take the n index of the "locations precipitation" table 

n = 1
print('name of mnt range:', hma_df.Name[n])

point_lat = locations_pr.latitude[n]
point_lon = locations_pr.longitude[n]

name of mnt range: Western Himalaya


In [6]:
# %%time

# kick out unnecessary dimention:
temps = temps.sel(expver= slice(0, 1))
precip = precip.sel(expver = slice(0,1))
clouds = clouds.sel(expver = slice(0,1))
radiation = radiation.sel(expver = slice(0,1))

# select the point in the each dataset 

temps_point = temps.sel(latitude = point_lat, longitude = point_lon, method = 'nearest')
precip_point = precip.sel(latitude = point_lat, longitude = point_lon, method = 'nearest')
clouds_point = clouds.sel(latitude = point_lat, longitude = point_lon, method = 'nearest')
radiation_point = radiation.sel(latitude = point_lat, longitude = point_lon, method = 'nearest')


In [7]:
# %%time
# ----------- convert to dataframe and cleanup: ----------- 

# temps_point_df = temps_point.to_dataframe()

In [8]:
# temps_point
len(temps.time.values)

368184

## workapathround (make netcdf conversion to df faster): 

### (1) temperatures 

In [11]:
path_temp = '/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/2m-temperature/'

# precipitations
path_precip = '/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/total-precipitation/'

# total cloud cover 
path_clouds = '/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/total-cloud-cover/'

# incoming radiation 
path_radiation = '/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/surface-solar-radiation-downwards/'


# path for output timeseries
path_output_temp = "/Users/varyabazilova/Desktop/hma_regions/out/timeseries"

In [12]:
# take the n index of the "locations precipitation" table 

n = 1
print('name of mnt range:', hma_df.Name[n])

point_lat = locations_pr.latitude[n]
point_lon = locations_pr.longitude[n]

name of mnt range: Western Himalaya


In [14]:
%%time

# timeseries = xr.Dataset()

for n, f in enumerate(os.listdir(path_temp)):
    # read every file
    ds = xr.open_dataset(os.path.join(path_temp, f), decode_coords="all")
    # select coordinates: 
    ds = temps.sel(latitude = point_lat, longitude = point_lon, method = 'nearest')
    # kick out the expver dimention
    ds = ds.sel(expver= slice(0, 1))
    # convert to dataframe
    ds_df = ds.to_dataframe().dropna().drop_duplicates()
    # export:
    # 1. create a year 
    year = n + 1979
    # 2. save to csv
    ds_df.to_csv(path_output_temp + '_temp_{y}.csv'.format(y=year))
    
    print('year - done:', year)
 


year - done: 1979
year - done: 1980
year - done: 1981
year - done: 1982


KeyboardInterrupt: 

In [ ]:
## read all the csv files created 

# append from the bottom 


In [ ]:
%%time

# timeseries = xr.Dataset()

for n, f in enumerate(os.listdir(path_precip)):
    # read every file
    ds = xr.open_dataset(os.path.join(path_precip, f), decode_coords="all")
    # convert to dataframe
    ds_df = ds.to_dataframe().dropna().drop_duplicates()
    # export:
    # 1. create a year 
    year = n + 1979
    # 2. save to csv
    ds_df.to_csv(path_output + '_precip_{y}.csv'.format(y=year))
    
    print('year - done:', year)
 



In [ ]:
%%time

# timeseries = xr.Dataset()

for n, f in enumerate(os.listdir(path_clouds)):
    # read every file
    ds = xr.open_dataset(os.path.join(path_clouds, f), decode_coords="all")
    # convert to dataframe
    ds_df = ds.to_dataframe().dropna().drop_duplicates()
    # export:
    # 1. create a year 
    year = n + 1979
    # 2. save to csv
    ds_df.to_csv(path_output + '_clouds_{y}.csv'.format(y=year))
    
    print('year - done:', year)
 



In [ ]:
%%time

# timeseries = xr.Dataset()

for n, f in enumerate(os.listdir(path_radiation)):
    # read every file
    ds = xr.open_dataset(os.path.join(path_radiation, f), decode_coords="all")
    # convert to dataframe
    ds_df = ds.to_dataframe().dropna().drop_duplicates()
    # export:
    # 1. create a year 
    year = n + 1979
    # 2. save to csv
    ds_df.to_csv(path_output + '_SWradiation_{y}.csv'.format(y=year))
    
    print('year - done:', year)
 


